## Gesture Kinematic Analysis


In this tutorial, we learn how to compute the kinematic features (speed, velocity, acceleration and jerk) for gestures using Mediapipe keypoints.


## Script Overview


 - Import necessary packages
 - Read saved Mediapipe keypoints 
 - Normalise the keypoints
 - Perform smoothing
 - Extract the speed, velocity, acceleration, and jerk
 - Visualise the kinematic measures


Citation: Pouw, W. (2023). Selecting, smoothing, and deriving measures from motion tracking, and merging with acoustics and annotations. Retrieved from: https://envisionbox.org/embedded_MergingMultimodal_inPython.html


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import plotly.graph_objects as go
from scipy.signal import butter, filtfilt

from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook


In [ ]:
data = pd.read_csv('sample1_body.csv')
data.head()


## Normalisation 


Normalization of keypoints is important for ensuring a consistent representation of poses across different frames and individuals. Moreover, it addresses the potential variance in the actual position of a person in the image, directing the pose representation to emphasize the relative positions of body parts rather than their absolute positions.


The following code normalizes a set of keypoints representing a skeleton by centering them around the mean position between the left and right shoulders and scaling them based on the distance between these shoulders. 


In [ ]:
def normalize_skeleton_landmarks(keypoints):
    left_shoulder, right_shoulder = 11, 12  # Assuming the indices for left and right shoulder in markersbody
    mid = keypoints[:, [left_shoulder, right_shoulder], :].mean(axis=1, keepdims=True)

    shoulder_length = np.linalg.norm(keypoints[:, left_shoulder, :] - keypoints[:, right_shoulder, :], ord=2, axis=1)
    normalized_keypts = (keypoints - mid) / shoulder_length[:, None, None]
    return normalized_keypts


In [ ]:
markersbody = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
          'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 
          'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
          'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
          'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']

xyz_columns = [axis + '_' + landmark  for landmark in markersbody for axis in ['X', 'Y', 'Z']]


# Select only the columns representing X, Y, Z coordinates
keypoints_columns = data[xyz_columns]


# Take the absolute value of Z coordinates
z_columns = [col for col in keypoints_columns.columns if col.startswith('Z_')]

# Reshape the data into the required format (num_frames, num_keypoints, 3)
num_keypoints = len(xyz_columns) // 3
keypoints_array = keypoints_columns.values.reshape(-1, num_keypoints, 3)

# Apply normalization function
normalized_keypoints = normalize_skeleton_landmarks(keypoints_array)

# Update the DataFrame with the normalized values
normalized_keypoints_df = pd.DataFrame(normalized_keypoints.reshape(-1, num_keypoints * 3), columns=xyz_columns)


## Data Visualisation
